## NumPy: the Dark Side and Its Applications

![numpy-logo](images/NumPy_logo_2020.svg)

**Dboy Liao**

[Medium](https://medium.com/@dboyliao)
[GitHub](https://github.com/dboyliao)
[LinkedIn](https://www.linkedin.com/in/yin-chen-liao-69967188/)
[CakeResume](https://www.cakeresume.com/dboyliao)

<a href="https://github.com/dboyliao/TaipeiPy-numpy-talk-2023-Jan" target="_blank" style="position: absolute; top: 0; right: 0; z-index: 200;">
    <img decoding="async" loading="lazy" width="149" height="149" src="https://github.blog/wp-content/uploads/2008/12/forkme_right_darkblue_121621.png?resize=200%2C200" class="attachment-full size-full" alt="Fork me on GitHub" data-recalc-dims="1">
</a>

# The NumPy `ndarray`

```cpp
int matrix[3][5];
```

- arbitrary shape?

- flexible reshape?

- different data type?

![no-cpp](images/no_cpp.jpg)

```python
matrix = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15]
]
```

- 多維度
  - 陣列的陣列
  - 陣列的陣列的陣列
  - 陣列的陣列的陣列的...(下略 1000 字)

- 所有元素 (element) 屬於同一種資料型態 (data type)
  - `int`
  - `float32`
  - `float64`
  - `object`

來寫個 `reshape` 吧!

```python
def reshape_(in_array: list, new_shape):
    # 要怎麼決定 in_array 的 shape?
    # 要怎麼檢查裡面的資料型態?
    # 要怎麼檢查 new_shape 是合理的?
    ...
```

```python
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8, 9],
    [10, 11, 12, '13']
]
```

![better-way](images/better_way.jpg)

In [1]:
import numpy as np

array = np.arange(16, dtype=np.int8).reshape(4, 4).copy()
array

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]], dtype=int8)

In [2]:
array.shape

(4, 4)

In [3]:
array.strides

(4, 1)

In [4]:
array.data

In [5]:
array.base is None

True

![array-2d](images/array_2d.drawio.svg)

![array-2d-flatten](images/array_2d_flatten.drawio.svg)

In [6]:
arr_flatten = array.ravel()
arr_flatten

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int8)

```python
# Dark Magic 
np.lib.stride_tricks
```

那我們先來看看用這個 Dark Magic 可以怎麼做一個自己的 `reshape`

In [7]:
def flat_list(ll, acc=None):
    if acc is None:
        acc = []
    for l in ll:
        if not isinstance(l, list):
            acc.append(l)
        else:
            acc = flat_list(l, acc)
    return acc

In [8]:
flat_list([[1, 2, 3], [4, 5, 6]])

[1, 2, 3, 4, 5, 6]

In [9]:
flat_list([[1, 2, 3], [4, 5, 6], [1, 2, [3, 4]]])

[1, 2, 3, 4, 5, 6, 1, 2, 3, 4]

In [10]:
def reshape_(in_array, new_shape):
    flat_array = flat_list(in_array)
    new_strides = []
    acc = 1
    for s in new_shape[::-1]:
        new_strides.insert(0, acc*8)
        acc *= s
    return np.lib.stride_tricks.as_strided(
        flat_array,
        shape=new_shape,
        strides=new_strides
    ).tolist()

In [11]:
reshape_(
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15]
    ],
    (5, 3)
)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]

In [12]:
reshape_(
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15]
    ],
    (5, 3, 1)
)

[[[1], [2], [3]],
 [[4], [5], [6]],
 [[7], [8], [9]],
 [[10], [11], [12]],
 [[13], [14], [15]]]

In [13]:
reshape_(
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15]
    ],
    (6, 3)
)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [0, 3, 1]]

# NumPy is All You Need

###### Nature 2020 Review Paper

![numpy-nature](images/numpy-nature.webp)

[source](https://www.nature.com/articles/s41586-020-2649-2)

- strides, shape and tensor rank

- indexing
    - basic indexing
    - advanced indexing

- broadcasting

- vectorization

[Official Doc](https://numpy.org/doc/stable/user/basics.indexing.html)

![numpy-memorize](images/numpy-memorize.jpg)

![better-way](images/better_way.jpg)

![array-2d-flatten](images/array_2d_flatten.drawio.svg)

In [14]:
print("1 == 4 * 0 + 1 * 1:", 1 == 4*0 + 1*1)
print("6 == 4 * 1 + 1 * 2:", 6 == 4*1 + 1*2)

1 == 4 * 0 + 1 * 1: True
6 == 4 * 1 + 1 * 2: True


In [15]:
print("arr_flatten[1] == array[0, 1]:", arr_flatten[1] == array[0, 1])
print("arr_flatten[6] == array[1, 2]:", arr_flatten[6] == array[1, 2])

arr_flatten[1] == array[0, 1]: True
arr_flatten[6] == array[1, 2]: True


In [16]:
array.strides

(4, 1)

- linear offset: the offset of an element in the flattened array
- $\mathbf{arr}$: a m-dims array
    - strides: $(s_0, s_1, ..., s_{m-1})$
    - shape: $(d_0, d_1, ..., d_{m-1})$
- $e = \mathbf{arr}[i_0, i_1, ..., i_{m-1}]$, element in $\mathbf{arr}$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \text{offset}_e = \sum\limits_{j=0}^{m-1} s_j \cdot i_j
$$

# Applications

## Shared Memory View

In [17]:
cube = np.arange(3*3*3).reshape((3, 3, 3)).copy()
cube

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

![array-3d](images/array-3d.drawio.svg)

![array-3d-strided](images/array-3d-strided.drawio.svg)

In [18]:
strided_cube = cube[::2, ::2, ::2]
strided_cube

array([[[ 0,  2],
        [ 6,  8]],

       [[18, 20],
        [24, 26]]])

- linear offset: the offset of an element in the flattened array
- $\mathbf{arr}$: a m-dims array
    - strides: $(s_0, s_1, ..., s_{m-1})$
    - shape: $(d_0, d_1, ..., d_{m-1})$
- $e = \mathbf{arr}[i_0, i_1, ..., i_{m-1}]$, element in $\mathbf{arr}$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \text{offset}_e = \sum\limits_{j=0}^{m-1} s_j \cdot i_j
$$

In [19]:
cube.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26])

In [20]:
cube.shape

(3, 3, 3)

In [21]:
cube.strides # number of bytes

(72, 24, 8)

- $\mathbf{cube}$: a 3-dims array
    - strides: $(72, 24, 8)$
    - shape: $(3, 3, 3)$
- $e = \mathbf{cube}[i_0, i_1, i_2]$, element in $\mathbf{cube}$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \text{offset}_e = 72 \cdot i_0 + 24 \cdot i_1 + 8 \cdot i_2
$$

In [22]:
strided_cube.ravel()

array([ 0,  2,  6,  8, 18, 20, 24, 26])

In [23]:
strided_cube.shape

(2, 2, 2)

In [24]:
strided_cube.strides

(144, 48, 16)

- $\mathbf{strided\_cube}$: a 3-dims array
    - strides: $(144, 48, 16)$
    - shape: $(2, 2, 2)$
- $\mathbf{strided\_cube}$$[i_0^{\prime}, i_1^{\prime}, i_2^{\prime}]$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \begin{align*}
    \text{offset}_e &= 144 \cdot i_0^{\prime} + 48 \cdot i_1^{\prime} + 16 \cdot i_2^{\prime} \\
                    &= 72 \cdot (2 \cdot i_0^{\prime}) + 24 \cdot (2 \cdot i_1^{\prime}) + 8 \cdot (2 \cdot i_2^{\prime}) \\
                    &= 72 \cdot i_0 + 24 \cdot i_1 + 8 \cdot i_2
    \end{align*}
$$

![array-3d-strided](images/array-3d-strided.drawio.svg)

In [25]:
strided_cube.data, type(strided_cube.data)

(<memory at 0x105ea31f0>, memoryview)

[`memoryview` documentation](https://docs.python.org/3/library/stdtypes.html#memoryview)

In [26]:
import struct

data_bytes = strided_cube.data.tobytes()
for i in range(8):
    elem = struct.unpack(
        'q',
        data_bytes[i*strided_cube.itemsize:(i+1)*strided_cube.itemsize],
    )[0]
    print(elem, end=" ")

0 2 6 8 18 20 24 26 

In [27]:
np.lib.stride_tricks.as_strided(
    cube,
    strides=(144, 48, 16),
    shape=(2, 2, 2)
)

array([[[ 0,  2],
        [ 6,  8]],

       [[18, 20],
        [24, 26]]])

In [28]:
strided_cube.base is cube

True

![array-3d-not-strided](images/array-3d-not-strided.drawio.svg)

In [29]:
# advanced indexing
random_cube = cube[
    [0, 0, 0, 0, 2, 2, 2, 2],
    [0, 0, 2, 2, 0, 0, 2, 2],
    [0, 2, 1, 2, 0, 1, 0, 2]
]
random_cube.reshape((2, 2, 2))

array([[[ 0,  2],
        [ 7,  8]],

       [[18, 19],
        [24, 26]]])

In [30]:
random_cube.base is None

True

In [31]:
cube.ravel().base is cube

True

In [32]:
cube.flatten().base is cube

False

![ok-then-what](images/okey_then_what.jpg)

### Time Series: Sliding Window with Shared-Memory View

In [33]:
data = np.arange(20, dtype=np.int8)
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int8)

In [34]:
windows_ = np.empty_like(data, shape=(16, 5))
for i in range(16):
    windows_[i, :] = data[i:(i+5)]
windows_

array([[ 0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5],
       [ 2,  3,  4,  5,  6],
       [ 3,  4,  5,  6,  7],
       [ 4,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  9],
       [ 6,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14],
       [11, 12, 13, 14, 15],
       [12, 13, 14, 15, 16],
       [13, 14, 15, 16, 17],
       [14, 15, 16, 17, 18],
       [15, 16, 17, 18, 19]], dtype=int8)

![better-way](images/better_way.jpg)

In [35]:
windows = np.lib.stride_tricks.as_strided(
    data,
    shape=(16, 5),
    strides=(1, 1)
)

In [36]:
windows

array([[ 0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5],
       [ 2,  3,  4,  5,  6],
       [ 3,  4,  5,  6,  7],
       [ 4,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  9],
       [ 6,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14],
       [11, 12, 13, 14, 15],
       [12, 13, 14, 15, 16],
       [13, 14, 15, 16, 17],
       [14, 15, 16, 17, 18],
       [15, 16, 17, 18, 19]], dtype=int8)

In [37]:
X = windows[:, :4]
Y = windows[:, 4]

In [38]:
X

array([[ 0,  1,  2,  3],
       [ 1,  2,  3,  4],
       [ 2,  3,  4,  5],
       [ 3,  4,  5,  6],
       [ 4,  5,  6,  7],
       [ 5,  6,  7,  8],
       [ 6,  7,  8,  9],
       [ 7,  8,  9, 10],
       [ 8,  9, 10, 11],
       [ 9, 10, 11, 12],
       [10, 11, 12, 13],
       [11, 12, 13, 14],
       [12, 13, 14, 15],
       [13, 14, 15, 16],
       [14, 15, 16, 17],
       [15, 16, 17, 18]], dtype=int8)

In [39]:
Y

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
      dtype=int8)

In [40]:
X.base is windows, Y.base is windows

(True, True)

In [41]:
X_ = windows[:, [0, 1, 2, 3]] # X = windows[:, :4]

In [42]:
X_.base is windows

False

basic indexing v.s advanced indexing

## Nested Loop and Vectorization (Broadcasting)

- given an array `a`, which of shape `(20, 5)`
- given an array `b`, which of shape `(30, 5)`
- given an array `c`, which of shape `(50, 1)`

create an array `out` of shape `(20, 30, 50)`, where `out[i, j, k]` is given as following

```python
out[i, j, k] = sum([c[k] if a_ > b_ else 0.0 for a_, b_ in zip(a[i], b[j])])
```

In [43]:
Na, Nb, Nc = 20, 30, 50
a = np.random.rand(Na, 5)
b = np.random.rand(Nb, 5)
c = np.random.rand(Nc, 1)

In [44]:
%%time
out = np.zeros((Na, Nb, Nc), dtype=float)
for i in range(Na):
    for j in range(Nb):
        for k in range(Nc):
            out[i, j, k] = ((a[i] > b[j]) * c[k]).sum()

CPU times: user 186 ms, sys: 22 ms, total: 208 ms
Wall time: 186 ms


![better-way](images/better_way.jpg)

In [45]:
%%time
out_ = (
    (
        a[:, np.newaxis, np.newaxis, :] > b[np.newaxis, :, np.newaxis, :]
    ) * c[np.newaxis, np.newaxis, :, :]
).sum(axis=-1)
out_.shape

CPU times: user 1.81 ms, sys: 1.27 ms, total: 3.09 ms
Wall time: 2.16 ms


(20, 30, 50)

In [46]:
np.allclose(
    out,
    out_
)

True

In [47]:
print(a.shape, a[:, np.newaxis, np.newaxis, :].shape)

(20, 5) (20, 1, 1, 5)


In [48]:
a[:, np.newaxis, np.newaxis, :].strides

(40, 0, 0, 8)

```python
out = np.zeros((Na, Nb, Nc), dtype=float)
for i in range(Na):
    for j in range(Nb):
        for k in range(Nc):
            out[i, j, k] = ((a[i] > b[j]) * c[k]).sum()
```

<br/>

```python
out_ = (
    (
        a[:, np.newaxis, np.newaxis, :] > b[np.newaxis, :, np.newaxis, :]
    ) * c[np.newaxis, np.newaxis, :, :]
).sum(axis=-1)
```

# Take Home Messages

- the foundation data structure of `numpy`: `ndarray`
- shared-memory view
- indexing
- broadcasting

**NEVER** write nested loops ever again

At any time, think of this picture and this talk when you want to apply nested loops on `ndarray`s:

![better-way](images/better_way.jpg)

# What's Missing in This Talk

- `ndarray.flags` and more other interesting attributes
- row-major v.s column-major array
  - the memory layout of the array
  - the impact to the performance
- sparse array

# Learning Resources


- https://github.com/wadetb/tinynumpy
  - pure python, `numpy` compliant implementation
- https://github.com/dboyliao/numPY
  - my work
  - try to build a pure python implementation of `numpy`
  - education purpose
- https://github.com/rougier/numpy-100
  - 100 numpy exercises with solutions